In [1]:
from Multi_Fair_Boosting import Multi_Fair as maximus
from sklearn.model_selection import StratifiedShuffleSplit as ss
from DataPreprocessing.my_utils import get_score,get_fairness,vis

In [2]:
 def train_classifier1(X_train, X_test, y_train, y_test, sa_index, p_Group, base_learners,preference):
    
    classifier = maximus(n_estimators=base_learners, saIndex=sa_index, saValue=p_Group,preference=None,pareto=True)
    
    classifier.fit(X_train, y_train)
    
    y_pred_probs = classifier.predict_proba(X_test)[:, 1]
    y_pred_labels = classifier.predict(X_test)
    f=classifier.feature_importances_
    #return classifier.conf_scores, classifier.get_weights_over_iterations(), classifier.get_initial_weights()
    return y_pred_probs, y_pred_labels,classifier


In [3]:
def train(X,y):
    in_ts,pred1,fx=[],[],[]
    sss = ss(n_splits=5,test_size=0.4)    
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        in_ts.append(test_index)    
        pb1,pd1,f1=train_classifier1(X_train, X_test, y_train, y_test, sa_index,     p_Group, 499,preference=None)
        pred1.append(pd1)
        fx.append(f1)
        print(f1.theta-1," : ",f1.ob[f1.theta-1])
    return in_ts,pred1,fx

In [4]:
results,performance,Hx=[],[],[]
for dt in ['Compas']:
    if dt=='Adult':
        from DataPreprocessing.load_adult import load_adult
        X, y, sa_index, p_Group, x_control,F = load_adult()
        #v='Adult_2_sensi_Mari_Sex'
        saf=sa_index[1]
    elif dt=='Bank':
        from DataPreprocessing.load_bank import load_bank
        X, y, sa_index, p_Group, x_control,F = load_bank()
        saf=sa_index[0]
        print(saf)
    elif dt=='Credit':
        from DataPreprocessing.load_credit import load_credit
        X, y, sa_index, p_Group, x_control,F = load_credit()
        saf=sa_index[0]
    elif dt=='Compas':
        from DataPreprocessing.load_compas_data import load_compas
        X, y, sa, p_G, x_control,F = load_compas()
        sa_index=[sa[-1],sa[0]]
        p_Group=[p_G[-1],p_G[0]]
    sensitives=[F[v] for v in sa_index]
    in_ts,pred1,f1=train(X,y)
    results.append(list(get_fairness(sa_index,p_Group,in_ts,pred1,X,y).values()))
    performance.append(get_score(pred1,in_ts,X,y))
    Hx.append(f1)

Features we will be using for classification are: ['age_cat_25 - 45', 'age_cat_Greater than 45', 'age_cat_Less than 25', 'race', 'sex', 'priors_count', 'c_charge_degree', 'target'] 

105  :  [0.34491472 0.00834023 0.04984424]
105  :  [0.33891346 0.01602889 0.08934119]
495  :  [0.34523057 0.01388557 0.03871068]
431  :  [0.33733418 0.06429419 0.03356974]
79  :  [0.33765003 0.0567521  0.06377027]


For Sensitive attribute index  4
avg_TPR_unprot: 0.6580431410512911 avg_TPR_prot: 0.664430449451047
avg_TNR_unprot: 0.6496822060638212 avg_TNR_prot: 0.6578987548714973
avg_Maximum_Mistreatment: 0.05314702266867386

-------------------------------------------



For Sensitive attribute index  3
avg_TPR_unprot: 0.6322415561577047 avg_TPR_prot: 0.6728323795949164
avg_TNR_unprot: 0.6405839546188818 avg_TNR_prot: 0.660952818932729
avg_Maximum_Mistreatment: 0.05059890969283347

-------------------------------------------

Multi_Max_Mistreatment: 0.05314702266867386
avg_TPR: 0.6591549295774647 avg_TNR